## MovieAssignment Group 6  -By (Harsh, Yash, Kaushik, Deshna)

## Sort_scene_images
 We are using this funtion for sorting the images in order so that the scene can take the images dynamically in order.

In [1]:
from PIL import Image
import os

# Function for sorting given scene image files so that they display in comic book as intended
def sort_scene_images(scene_dir):
    return [f'{num}.jpeg' for num in sorted([int(img_str.split(".")[0]) for img_str in os.listdir(scene_dir) if img_str.split(".")[1]== "jpeg" or img_str.split(".")[1]== "jpg"])]

## Imgcompress_mem
We are reading the image then resizing it and after calculating the new dimensions we are compressing it 

In [2]:
import cv2
def imgcompress_mem(path_in, k):
    img = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)

    # set the ratio of resized image
    width = int((img.shape[1])/k)
    height = int((img.shape[0])/k)

    # resize the image by resize() function of openCV library
    return cv2.resize(img, (width, height), interpolation=cv2.INTER_AREA)

## Cartoonizeble_mem
After compressing the image we are adding blur and border to image so that the resulting image looks cartoonized 

In [3]:
# Controls blur amount and line size
def cartoonizebl_mem(path_in, k, blur, line):
    
    imgc = imgcompress_mem(path_in, k)
    #imgc_pil = cv2.cvtColor(imgc, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    #display(Image.fromarray(imgc_pil))

    line_size = line
    blur_value = blur
    #imgc = cv2.imread(path_out, cv2.IMREAD_UNCHANGED)
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, blur_value)
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
    bigedges_pil = cv2.cvtColor(bigedges, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    #display(Image.fromarray(bigedges_pil))

    return cv2.bitwise_and(imgc, imgc, mask=bigedges)

## Translation Library
We are using deep_translator to translate the code into different languages.

In [4]:
pip install deep_translator

Note: you may need to restart the kernel to use updated packages.


## Translate_text
This function uses the deep_translator library to translate input text line by line into either Hindi or Traditional Chinese, based on the selected language. It splits the text into lines, translates each line, and then joins the translated lines into a single string.

In [5]:
from deep_translator import GoogleTranslator

def translate_text(text, language):
    # Check which language the user wants to translate to and set the target language code
    if language == "hindi":
        target = "hi"  # Hindi language code
    elif language == "chinese":
        target = "zh-TW"  # Traditional Chinese language code
    
    # Split the input text by new lines
    lines = text.split('\n')

    # Initialize the translator with the source language set to auto-detect and the target language
    translator = GoogleTranslator(source='auto', target=target)
    
    # Translate each line and store the results in a list
    translated_lines = [translator.translate(line) for line in lines]
    
    # Join the translated lines back into a single string with new lines and return it
    return '\n'.join(translated_lines)

## add_dialogue_and_cartoonise
This function takes an image, applies a cartoon effect, adds a border, and overlays translated dialogue based on the selected language (Hindi, Chinese, or English). It creates a text box for the dialogue with semi-transparency and ensures the text is displayed in the appropriate font. Finally, the image is saved in a language-specific folder, creating the directory if it doesn't already exist.

In [6]:
from PIL import ImageOps, ImageDraw, ImageFont 

# Function that adds dialogue and cartoonizes an image
def add_dialogue_and_cartoonise(scene_dir, image, dialogue, language):

    # Call cartoonizebl_mem function to cartoonize the image and convert it to PIL format
    cblimg = cartoonizebl_mem(f'{scene_dir}/{image}', 9, 11, 13)
    cblimg_pil = cv2.cvtColor(cblimg, cv2.COLOR_BGR2RGB)

    # Adding a border around the image
    img_border = ImageOps.expand(Image.fromarray(cblimg_pil), border=(30, 30), fill=(0, 0, 0))

    # Set dialogue overlay properties
    TINT_COLOR = (0, 0, 0)  # Black overlay for dialogue box
    TRANSPARENCY = .4  # 40% transparency
    OPACITY = int(255 * TRANSPARENCY)  # Convert transparency to opacity

    # Convert image to RGBA (with alpha channel) to handle transparency
    img = img_border.convert('RGBA')

    # Create an overlay for the text box
    overlay = Image.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = ImageDraw.Draw(overlay)

    # Set font based on the selected language
    if language == "hindi":
        font = ImageFont.truetype("TiroDevanagariHindi-Regular.ttf", 30)
    elif language == "chinese":
        font = ImageFont.truetype("NotoSansTC-Bold.ttf", 30)
    else:
        font = ImageFont.truetype("Ubuntu-Bold.ttf", 30)

    # Calculate text dimensions for placement
    text = dialogue
    left, top, right, bottom = font.getbbox(text)
    w = right - left
    h = bottom - top

    # Replace newline character for multi-line text
    text = text.replace('\\n', "\n")

    # Draw a rectangle for each line of text
    for index in range(len(text.split("\n")), 0, -1):
        x, y = 30, img.height - index*h - 40
        draw.rectangle((x, y, x + img.width - 60, y + index*h + 10), fill=TINT_COLOR+(OPACITY,))

    # Translate the text if the language is Hindi or Chinese
    if language == "hindi":
        text = translate_text(text, language)
    elif language == "chinese":
        text = translate_text(text, language)

    # Draw the translated text on the image
    draw.text((30, img.height - len(text.split("\n"))*h - 40), text, fill=(209, 239, 8), font=font)

    # Combine the image and the overlay
    img = Image.alpha_composite(img, overlay)
    img = img.convert("RGB")  # Convert to RGB to save as JPG

    # Save the image in the correct directory based on the language
    try:
        if language == "hindi":
            img.save(f'{scene_dir}ModifiedWithDialogueHindi/{image}')
        elif language == "chinese":
            img.save(f'{scene_dir}ModifiedWithDialogueChinese/{image}')
        else:
            img.save(f'{scene_dir}ModifiedWithDialogue/{image}')
    except FileNotFoundError:
        # If the directory doesn't exist, create it and save the image
        if language == "hindi":
            os.mkdir(f'{scene_dir}ModifiedWithDialogueHindi')
            img.save(f'{scene_dir}ModifiedWithDialogueHindi/{image}')
        elif language == "chinese":
            os.mkdir(f'{scene_dir}ModifiedWithDialogueChinese')
            img.save(f'{scene_dir}ModifiedWithDialogueChinese/{image}')
        else:
            os.mkdir(f'{scene_dir}ModifiedWithDialogue')
            img.save(f'{scene_dir}ModifiedWithDialogue/{image}')

## scene_to_img
This function processes scene images by adding cartoon effects and dialogue in the specified language, then combines the images into a single page by stacking them horizontally and vertically. The images are resized to maintain consistent dimensions, and the final combined image is saved in a language-specific folder. If the folder doesn't exist, it's created before saving.

In [7]:
import numpy as np
import pandas as pd

# Function that combines multiple images of a scene into one single page image
def scene_to_img(scene_dir, cols, language="english"):
    
    # Sort the images in the scene directory
    scene = sort_scene_images(scene_dir)
    
    # Load the dialogue data from a CSV file
    data = pd.read_csv(f'{scene_dir}/dialogues.csv', names=["English"], encoding='cp1252')
    
    # Add dialogue to each image and apply cartoonization
    for index, img in enumerate(scene):
        add_dialogue_and_cartoonise(scene_dir, img, data.values[index][0], language)
    
    # Open images after processing, depending on the language specified
    if language == "hindi":
        imgs = [Image.open(f'{scene_dir}ModifiedWithDialogueHindi/{img}') for img in sort_scene_images(f'{scene_dir}ModifiedWithDialogueHindi')]
    elif language == "chinese":
        imgs = [Image.open(f'{scene_dir}ModifiedWithDialogueChinese/{img}') for img in sort_scene_images(f'{scene_dir}ModifiedWithDialogueChinese')]
    else:
        imgs = [Image.open(f'{scene_dir}ModifiedWithDialogue/{img}') for img in sort_scene_images(f'{scene_dir}ModifiedWithDialogue')]

    # Group images into rows of 'cols' columns
    horizontal_image_rows = [imgs[i:i + cols] for i in range(0, len(imgs), cols)]
    vertical_image_cols = []

    # Resize images in each row to have the same height and combine them horizontally
    for imgs_row in horizontal_image_rows:
        min_shape = sorted([(np.sum(i.size), i.size ) for i in imgs_row])[0][1]  # Get the smallest image size
        resized_horizontal = [np.asarray( i.resize(min_shape) ) for i in imgs_row]  # Resize all images to the smallest size
        imgs_comb_horizontally = np.hstack(resized_horizontal)  # Stack images horizontally
        vertical_image_cols.append(Image.fromarray(imgs_comb_horizontally))  # Append the combined horizontal row to a list
    
    # Resize all combined horizontal rows to the same width and stack them vertically
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in vertical_image_cols])[0][1]  # Get the smallest height
    resized_vertical = [np.asarray( i.resize(min_shape) ) for i in vertical_image_cols]  # Resize all columns to the smallest height
    imgs_comb_vertically = np.vstack(resized_vertical)  # Stack all rows vertically

    # Convert the final image from a numpy array to a PIL Image object
    imgs_comb = Image.fromarray(imgs_comb_vertically)
    
    # Resize the final image to 1/8th of its original size
    imgs_comb.resize((imgs_comb.width//8, imgs_comb.height//8))

    # Save the final image in the appropriate directory, creating the directory if it doesn't exist
    try:
        if language == "hindi":
            imgs_comb.save(f'MoviePagesHindi/{scene_dir}.jpg')
        elif language == "chinese":
            imgs_comb.save(f'MoviePagesChinese/{scene_dir}.jpg')
        else:
            imgs_comb.save(f'MoviePages/{scene_dir}.jpg')
    except FileNotFoundError:
        # If the folder doesn't exist, create it and then save the image
        if language == "hindi":
            os.mkdir('MoviePagesHindi')
            imgs_comb.save(f'MoviePagesHindi/{scene_dir}.jpg')
        elif language == "chinese":
            os.mkdir('MoviePagesChinese')
            imgs_comb.save(f'MoviePagesChinese/{scene_dir}.jpg')
        else:
            os.mkdir('MoviePages')
            imgs_comb.save(f'MoviePages/{scene_dir}.jpg')


In [8]:
# Passing all the scenes to create pages for each scenes
scene_to_img("Scene1", 3)
scene_to_img("Scene2", 3)
scene_to_img("Scene3", 3)
scene_to_img("Scene4", 3)
scene_to_img("Scene5", 3)
scene_to_img("Scene6", 3)

## Generating PDF
Adding all the generated pages of the scenes in a pdf.

In [9]:
from fpdf import FPDF

# Set up the dimensions for the PDF pages (8.5 x 11 inches)
width, height = 8.5, 11
pdf = FPDF(unit = "in", format = [width, height])

# List all images in the 'MoviePages' directory
imagelist = os.listdir('MoviePages')

# List of signatures to be added to each page
ls = ["Harsh", "Yash", "Deshna", "Kaushik", "Harsh and Yash", "Kaushik and Deshna"]

# Loop through each image in the image list and add it to a new PDF page
for index, image in enumerate(imagelist):
    pdf.add_page()  # Create a new page
    # Place the image on the page with a slight margin at the bottom
    pdf.image(f'MoviePages/{image}', 0, 0, width, height - 0.25)
    
    # Set font for the signature text (Arial, Bold, size 11)
    pdf.set_font('Arial', 'B', 11)
    
    # Add a signature at the bottom of the page
    pdf.text(6, 10.92, f"Signed By {ls[index]}")

# Output the final PDF file
pdf.output("moviebook.pdf", "F")
print("done! you should see moviebook.pdf file that is generated using this code.")


done! you should see moviebook.pdf file that is generated using this code.


## For Hindi
Translation in Hindi generally takes time. Please be patient!

In [10]:
# Doing the same thing for translation in Hindi
scene_to_img("Scene1", 3, "hindi")
scene_to_img("Scene2", 3, "hindi")
scene_to_img("Scene3", 3, "hindi")
scene_to_img("Scene4", 3, "hindi")
scene_to_img("Scene5", 3, "hindi")
scene_to_img("Scene6", 3, "hindi")

In [11]:
from fpdf import FPDF

# Set the dimensions for the PDF pages (8.5 x 11 inches)
width, height = 8.5, 11
pdf = FPDF(unit = "in", format = [width, height])  # Initialize the PDF with the specified format

# List all images in the 'MoviePagesHindi' directory
imagelist = os.listdir('MoviePagesHindi')

# List of names for signing each page of the PDF
ls = ["Harsh", "Yash", "Deshna", "Kaushik", "Harsh and Yash", "Kaushik and Deshna"]

# Loop through each image in the imagelist and add it to a new page in the PDF
for index, image in enumerate(imagelist):
    pdf.add_page()  # Add a new page to the PDF
    
    # Insert the image onto the PDF page, leaving a small margin at the bottom
    pdf.image(f'MoviePagesHindi/{image}', 0, 0, width, height - 0.25)
    
    # Set the font for the signature text (Arial, Bold, size 11)
    pdf.set_font('Arial', 'B', 11)
    
    # Add the signature at the bottom right of the page
    pdf.text(6, 10.92, f"Signed By {ls[index]}")

# Save the generated PDF as 'moviebookHindi.pdf'
pdf.output("moviebookHindi.pdf", "F")
print("done! you should see moviebookHindi.pdf file that is generated using this code. Thank you for being patient. Enjoy the translation")

done! you should see moviebookHindi.pdf file that is generated using this code.


## For Chinese
Translation in Chinese takes more time. Please be more patient!

In [12]:
# Doing the same thing for translation in Chinese
scene_to_img("Scene1", 3, "chinese")
scene_to_img("Scene2", 3, "chinese")
scene_to_img("Scene3", 3, "chinese")
scene_to_img("Scene4", 3, "chinese")
scene_to_img("Scene5", 3, "chinese")
scene_to_img("Scene6", 3, "chinese")

In [13]:
from fpdf import FPDF

# Set up the dimensions for the PDF pages (8.5 x 11 inches)
width, height = 8.5, 11
pdf = FPDF(unit = "in", format = [width, height])  # Initialize the PDF with specified size

# Get a list of all images in the 'MoviePagesChinese' directory
imagelist = os.listdir('MoviePagesChinese')

# List of names for signing each page of the PDF
ls = ["Harsh", "Yash", "Deshna", "Kaushik", "Harsh and Yash", "Kaushik and Deshna"]

# Loop through each image and add it to the PDF
for index, image in enumerate(imagelist):
    pdf.add_page()  # Add a new page to the PDF
    
    # Insert the image on the page, leaving a small margin at the bottom
    pdf.image(f'MoviePagesChinese/{image}', 0, 0, width, height - 0.25)
    
    # Set the font for the signature (Arial, Bold, size 11)
    pdf.set_font('Arial', 'B', 11)
    
    # Add a signature at the bottom right of the page
    pdf.text(6, 10.92, f"Signed By {ls[index]}")

# Save the final PDF file as 'moviebookChinese.pdf'
pdf.output("moviebookChinese.pdf", "F")
print("done! you should see moviebookChinese.pdf file that is generated using this code. Thank you for being patient. Enjoy the translation")

done! you should see moviebookChinese.pdf file that is generated using this code.
